# Assignment 2: Emotion Classification

In [24]:
#all dependencies

import numpy as np
import pandas as pd
import seaborn as sns

import nltk
nltk.download('stopwords')

import tensorflow as tf

from sklearn.model_selection import train_test_split

[nltk_data] Downloading package stopwords to /Users/ygao/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [17]:
#constants
DATA_LOCATION = '../data/go_emotions_dataset.csv'
CLEAN_DATA_LOCATION = '../data/'
MODEL_LOCATION = '../model/'
RANDOM_SEED = 297

# Data preparation

We load the data from local source, and split it into train validation and test sets. 
Split corpus and target.

In [14]:
#load dataset
def load_data(filename):
    df = pd.read_csv(filename)
    df_raw_data = df.drop(columns=['id', 'example_very_unclear']) #text+categories
    category_columns = df_raw_data.columns[1:28] #extract categories
    df_data = pd.DataFrame({ #creating new df with 'text' and 'target
        'text': df_raw_data['text'],
        'target': df_raw_data[category_columns].apply(lambda row: row.values.tolist(), axis=1)
    })
    return df_data

df = load_data(DATA_LOCATION)

In [23]:
#split dataset into train, validation, test (70% 15% 15%)
#train: 147857 validation/test: 31684
df_train, df_temp = train_test_split(df, test_size=0.3, random_state=RANDOM_SEED)
df_val, df_test = train_test_split(df_temp, test_size=0.5, random_state=RANDOM_SEED)
#save the split data frames to csv just in case
df_train.to_csv(CLEAN_DATA_LOCATION+'raw_train.csv', index=False)
df_val.to_csv(CLEAN_DATA_LOCATION+'raw_val.csv', index=False)
df_test.to_csv(CLEAN_DATA_LOCATION+'raw_test.csv', index=False)

We have a closer look at the training set to see if the data is balanced. 

In [26]:
print("Possible sentiments are", np.unique(df_train['target']), "of", len(np.unique(df_train['target'])),"sentiments")
print("The number of posts for training is", len(df_train))
# size = len(corpus_train) + len(corpus_val) + len(corpus_test)
# print("Training corpus is " + str(int(100*np.sum(target_train)/len(target_train))) + "% positive reviews")
# info = pd.DataFrame([corpus_train[5]], columns=["raw text example"]) #use to track progress
# info

Possible sentiments are [list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
 list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])
 list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0])
 ...
 list([1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
 list([1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
 list([1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])] of 1700 sentiments
The number of posts for training is 147857


# Preprocessing